In [16]:
import os
import pandas as pd
from asr_decoders import ctc_segmentation_kurzinger as kurzinger_asr
import alignment_adagt as adagt

In [17]:
# Inputs
audioFile = '/vol/tensusers2/wharmsen/SERDA-data/round1/audio/words/segments/2RRDV-words_1_116-20230113140713310.wav'
promptDir = '/vol/tensusers2/wharmsen/SERDA-data/prompts/' #words_1-wordIDX.csv

In [18]:
"""
Function that extracts taskID from filename 
input: path to audio file with segment
output: words_1, words_2 or words_3
"""
def getTaskFromFilename(filename):
    return 'words_'+ os.path.basename(filename).split('-')[1].split('_')[1]


"""
Function that extracts wordID from filename
input: path to audio file with segment
output: 101, 102, 103, etc.
"""
def getWordIDFromFilename(filename):
    return os.path.basename(filename).split('-')[1].split('_')[2]

"""
Function that returns the prompted word based on wordID.
inputs:
promptsWordsDF  The DF containing all prompts and their IDs
wordID          The promptID

output:
prompt          The prompt
"""
def getPromptFromWordID(audioFile):
    task = getTaskFromFilename(audioFile)
    wordID = getWordIDFromFilename(audioFile)
    promptsWordsFile = task + '-wordIDX.csv'
    promptsWordsDF = pd.read_csv(os.path.join(promptDir, promptsWordsFile))
    return promptsWordsDF[promptsWordsDF['prompt_id'] == wordID]['prompt']

# Extract prompt from filename
prompt = getPromptFromWordID(audioFile)

In [19]:
# Get ASR result
audio_file = '/vol/tensusers2/wharmsen/SERDA-data/round1/audio/words/segments/2RRDV-words_1_116-20230113140713310.wav'
prompt = 'sprei'
cache_path = '/vol/tensusers5/wharmsen/wav2vec2/cache/' 
model = "fc" 
output_dir = "/vol/tensusers2/wharmsen/SERDA-experiment-data/round1/words/segments/ctc-kurz"
alignment, recognized = kurzinger_asr.get_asr_result(audio_file, prompt, cache_path, model, output_dir)

It is strongly recommended to pass the ``sampling_rate`` argument to this function. Failing to do so can result in silent errors that might be hard to debug.
It is strongly recommended to pass the ``sampling_rate`` argument to this function. Failing to do so can result in silent errors that might be hard to debug.


In [20]:
# Extract transcription from ASR result
transcription = " ".join([x['text'] for x in recognized])
transcription

'sprei nap prei'

In [25]:
# ADAPT alignment prompt - asrTranscription
outputDF = adagt.two_way_alignment(prompt, transcription) 
outputDF


,aligned_asrTrans,reversed_aligned_asrTrans,correct
prompt,,,
sprei,sprei nap prei,sprei nap prei,True


In [9]:
recognized

[{'text': 'sprei',
  'start': 0.010040160642570281,
  'end': 1.676706827309237,
  'conf': 0.0},
 {'text': 'nap',
  'start': 1.676706827309237,
  'end': 3.3132530120481927,
  'conf': 0.0},
 {'text': 'prei',
  'start': 3.3132530120481927,
  'end': 4.206827309236948,
  'conf': 0.0}]